## **Import Libraries**

In [1]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

## **Unzipping Dataset**

In [ ]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)
    

In [ ]:
import os
os.listdir('.')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.1,zoom_range=0.1,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

## **MODEL FOR BODY TYPE DETECTION**

In [ ]:
trainPath = '/home/wsuser/work/Dataset/body/training'
testPath = '/home/wsuser/work/Dataset/body/validation'

In [ ]:

training_set = train_datagen.flow_from_directory(trainPath,target_size=(244,244),batch_size=10,class_mode='categorical')
test_set = train_datagen.flow_from_directory(testPath,target_size=(244,244),batch_size=10,class_mode='categorical')

In [ ]:
training_set.class_indices

## **Declaring Model Variable**

In [ ]:
vgg=VGG16(input_shape=(244,244,3),weights='imagenet',include_top=False)



for layer in vgg.layers:
  layer.trainable=False


x=Flatten()(vgg.output)


prediction=Dense(3,activation='softmax')(x)


model=Model(inputs=vgg.input,outputs=prediction)
model.summary()


## **Compiling the Model**

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

## **Training model**

In [ ]:
r = model.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 25,
    steps_per_epoch=979//10,
    validation_steps = 171//10
)

In [ ]:
model.save('body.h5')

In [ ]:
!tar -zcvf body.tgz body.h5

In [ ]:
ls -1 

In [ ]:
!pip install watson-machine-learning-client --upgrade

## **Connecting with IBM CLOUD**

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {"url":"https://us-south.ml.cloud.ibm.com", "apikey":"apikey"}
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client, 'spacename')
#space_uid

In [ ]:
client.set.default_space(space_uid)

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
#software_spec_uid

In [ ]:
client.software_specifications.list()

In [ ]:
model_details = client.repository.store_model(model = 'body.tgz' , meta_props = {
    client.repository.ModelMetaNames.NAME : "body", 
    client.repository.ModelMetaNames.TYPE : "tensorflow_rt22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})
model_id = client.repository.get_model_id(model_details)

In [ ]:
client.repository.download(model_id, 'body_cloud.tar.gz')

In [ ]:
model_body = load_model('body.h5')

## **MODEL FOR LEVEL TYPE DETECTION**

In [ ]:
trainPath = '/home/wsuser/work/Dataset/level/training'
testPath = '/home/wsuser/work/Dataset/level/validation'

In [ ]:

training_set = train_datagen.flow_from_directory(trainPath,target_size=(244,244),batch_size=10,class_mode='categorical')
test_set = train_datagen.flow_from_directory(testPath,target_size=(244,244),batch_size=10,class_mode='categorical')

In [ ]:
training_set.class_indices

## **Declaring Model Variable**

In [ ]:
vgg=VGG16(input_shape=(244,244,3),weights='imagenet',include_top=False)



for layer in vgg.layers:
  layer.trainable=False


x=Flatten()(vgg.output)


prediction=Dense(3,activation='softmax')(x)


model1=Model(inputs=vgg.input,outputs=prediction)
model1.summary()

In [ ]:
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

## **Training model**

In [ ]:
y = model1.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 25,
    steps_per_epoch=979//10,
    validation_steps = 171//10
)

In [ ]:
model1.save('level.h5')

In [ ]:
!tar -zcvf level.tgz level.h5

In [ ]:
ls -1 

In [ ]:
model_details = client.repository.store_model(model = 'level.tgz' , meta_props = {
    client.repository.ModelMetaNames.NAME : "level", 
    client.repository.ModelMetaNames.TYPE : "tensorflow_rt22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})
model_id = client.repository.get_model_id(model_details)

In [ ]:
client.repository.download(model_id, 'level_cloud.tar.gz')

In [ ]:
model_body = load_model('level.h5')

In [ ]:
os.listdir('.')

In [ ]:
client.repository.download('model_id','body_cloud.tar.gz')

In [ ]:
client.repository.download('model1_id','level_cloud.tar.gz')